Create Test Data
===

Create test databases with fake data but matching the schema of the full databases on MSI.

Databases to mock:
 - Sites
 - Journal Metadata
 - Journal Text
 - Amps
 - Comments
 - Guestbooks
 
The only part of this notebook that requires being run on MSI is the copying of the schema directly from each prod database.  While this step could be skipped if we find ourselves generating new test data often, it has the benefit of ensuring that the dev/test database schema matches the prod database schema.

In [1]:
import os
import sqlite3
from datetime import datetime

In [2]:
SITE_DB = '/home/lana/shared/caringbridge/data/derived/site_metadata/site.sqlite'
JOURNAL_METADATA_DB = "/home/lana/shared/caringbridge/data/projects/recsys-peer-match/sqlite/journal_metadata.sqlite"
JOURNAL_TEXT_DB = '/home/lana/shared/caringbridge/data/projects/caringbridge_core/journal.sqlite'
COMMENTS_DB = '/home/lana/shared/caringbridge/data/projects/caringbridge_core/updated_comments.sqlite'
GUESTBOOK_DB = '/home/lana/shared/caringbridge/data/projects/caringbridge_core/guestbook.sqlite'
AMPS_DB = '/home/lana/shared/caringbridge/data/projects/recsys-peer-match/sqlite/amps.sqlite'

In [3]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

PosixPath('/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match')

In [4]:
prod_db_paths = [SITE_DB, JOURNAL_METADATA_DB, JOURNAL_TEXT_DB, COMMENTS_DB, GUESTBOOK_DB, AMPS_DB]
test_db_dir = os.path.join(git_root_dir, 'cb_web_app', 'flask_api', 'test', 'data')
assert os.path.exists(test_db_dir)
test_db_paths = [os.path.join(test_db_dir, os.path.basename(path)) for path in prod_db_paths]
test_db_paths

['/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/site.sqlite',
 '/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal_metadata.sqlite',
 '/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal.sqlite',
 '/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/updated_comments.sqlite',
 '/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/guestbook.sqlite',
 '/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/amps.sqlite']

In [5]:
for test_db_path in test_db_paths:
    if os.path.exists(test_db_path):
        os.remove(test_db_path)
print("Existing test databases deleted.")

Existing test databases deleted.


In [6]:
for prod_db_path, test_db_path in zip(prod_db_paths, test_db_paths):
    os.system(f"sqlite3 {prod_db_path} '.schema' | sqlite3 {test_db_path}")
    assert os.path.exists(test_db_path)

In [7]:
!du -h {test_db_dir}/*.sqlite

64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/amps.sqlite
64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/guestbook.sqlite
64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal_metadata.sqlite
64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal.sqlite
32K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/site.sqlite
96K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/updated_comments.sqlite


### Create helper functions for mock data generation

In [8]:
def get_db(db_filename):
    db = sqlite3.connect(
            db_filename,
            detect_types=sqlite3.PARSE_DECLTYPES
        )
    db.row_factory = sqlite3.Row
    return db

In [9]:
def insert_mock_site(db, site_id, title, description, healthCondition_category, visits, publish_date=None):
    if publish_date is None:
        publish_date = int(datetime.now().timestamp()) * 1000
    created_at = publish_date
    updated_at = publish_date
    
    db.execute('INSERT INTO site (site_id, title, description, healthCondition_category, age, ip, created_at, updated_at, publish_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (site_id, title, description, healthCondition_category, "adult", "127.0.0.1", created_at, updated_at, publish_date)
                )

In [10]:
def insert_mock_journal(db_meta, db_text, journal_oid, user_id, title, body, 
                        created_at=None, amp_count=0, photos_count=0, videos_count=0, platform="", lastEdit=0.0, is_nontrivial=True):
    site_id, site_index = journal_oid.split('-')
    site_id, site_index = int(site_id), int(site_index)
    if created_at is None:
        created_at = int(datetime.now().timestamp()) * 1000
    updated_at = created_at
    published_at = created_at
    
    db_meta.execute('INSERT INTO journal (site_id, journal_oid, user_id, created_at, updated_at, published_at, amp_count, photos_count, videos_count, platform, lastEdit, site_index, is_nontrivial) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
            (site_id, journal_oid, user_id, created_at, updated_at, published_at, amp_count, photos_count, videos_count, platform, lastEdit, site_index, is_nontrivial)
    )
    db_text.execute('INSERT INTO journalText (site_id, journal_oid, title, body) VALUES (?, ?, ?, ?)',
            (site_id, journal_oid, title, body)
    )
    

In [11]:
def insert_mock_comment(db, site_id, journal_oid, user_id, body_text, created_at=None):
    max_id = db.execute('SELECT MAX(CAST(comment_oid as integer)) as max_id FROM comments').fetchall()[0]['max_id']
    comment_oid = '0' if max_id is None else str(int(max_id) + 1)
    
    if created_at is None:
        created_at = int(datetime.now().timestamp()) * 1000
    updated_at = created_at
    
    db.execute('INSERT INTO comments (comment_oid, site_id, user_id, ancestor_id, ancestor_type, parent_id, parent_type, platform, isDeleted, body, signature, created_at, updated_at, lastEdit, amps_count, amps) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (comment_oid, site_id, user_id, journal_oid, 'journal', journal_oid, 'journal', "", 0, body_text, "", created_at, updated_at, "", 0, "")
                )

In [12]:
def insert_mock_guestbook(db, site_id, user_id, body_text, created_at=None,
                         platform="", isDeleted="", ip="127.0.0.1", fromTribute="", amps_count=0, amps=""):
    max_id = db.execute('SELECT MAX(CAST(gb_oid as integer)) as max_id FROM guestbook').fetchall()[0]['max_id']
    gb_oid = '0' if max_id is None else str(int(max_id) + 1)
    
    if created_at is None:
        created_at = int(datetime.now().timestamp()) * 1000
    updated_at = created_at
    
    db.execute('INSERT INTO guestbook (gb_oid, site_id, user_id, created_at, updated_at, body, platform, isDeleted, ip, fromTribute, amps_count, amps) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
                (gb_oid, site_id, user_id, created_at, updated_at, body_text, platform, isDeleted, ip, fromTribute, amps_count, amps)
                )

In [13]:
def insert_mock_amp(db, site_id, journal_oid, user_id, 
                    created_at=None, journal_user_id=0):    
    if created_at is None:
        created_at = int(datetime.now().timestamp()) * 1000
    updated_at = created_at
    
    db.execute('INSERT INTO amps (user_id, site_id, journal_oid, journal_user_id, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?)',
                (user_id, site_id, journal_oid, journal_user_id, created_at, updated_at)
                )

### Create Mock Data

In [14]:
# Create mock sites data
db = get_db(test_db_paths[0])
try:
    insert_mock_site(db, 1, 'Alisha Smith', 'Test site 1 description.', 'Cancer', 100)
    insert_mock_site(db, 2, 'Ben Smith', 
                     'Test site 2 description. About the length of a paragraph. Not too long, but probably long enough to need to be hidden behind a "Read more" fucntion. Does contain an emoji: 👀 The eyes emoji is a pair of eyes looking slightly to the side in most renditions of the emoji. However, when Google released the eyes emoji, their version only featured one eye. The eyes emoji was added as part of Unicode 6.0 in 2010 and added to Emoji 1.0 in 2015.',
                     'Condition Unknown', 2000)
    insert_mock_site(db, 3, 'Carla Smith', 
                     'Test site 3 description. Long.\n\nLorem ipsum dolor sit amet, consectetur adipiscing elit. Sed sollicitudin lacus eu fringilla fermentum. Nulla facilisi. Donec condimentum leo quis augue varius, sit amet imperdiet nibh consequat. Suspendisse ac ligula a urna tempor tincidunt eget eu massa. Sed vel placerat augue. Morbi a ultrices nulla. Quisque vulputate consequat aliquam. Nam sed ligula id urna varius ultrices. Donec consequat ex sed tortor mattis, eget tincidunt risus mattis.\n\nVivamus in blandit sem, congue rutrum lacus. Mauris eget ipsum dui. Mauris volutpat scelerisque viverra. Morbi non elit sit amet augue pretium lobortis eu at tortor. Quisque vitae eros nec diam tempus accumsan non in velit. Etiam vehicula pulvinar nisi vitae vehicula. Sed tincidunt eu libero sit amet pellentesque. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Nam eu justo dictum, accumsan nibh volutpat, varius nisi. Aliquam cursus ex vitae pretium eleifend. Nullam enim enim, iaculis ut blandit in, bibendum a diam.\n\nPraesent facilisis ex volutpat, egestas ante a, volutpat ipsum. Morbi aliquam dolor a sodales ultricies. Quisque scelerisque venenatis auctor. Praesent felis leo, cursus non velit vel, efficitur sollicitudin massa. Cras lobortis sollicitudin egestas. Phasellus quam odio, elementum a volutpat nec, rhoncus at nisl. Proin sed hendrerit nulla. Sed fringilla metus vel dolor euismod rhoncus. Ut non venenatis odio.\n\nEtiam metus eros, pellentesque laoreet magna et, tincidunt iaculis justo. Nam cursus et sapien eget aliquet. In diam nibh, faucibus imperdiet felis et, vehicula tincidunt leo. Mauris sodales, arcu in mollis pulvinar, tortor tellus dictum diam, id ullamcorper mauris ex id diam. Nam finibus justo id urna tristique, quis vulputate urna euismod. Morbi finibus laoreet viverra. Maecenas dictum fringilla erat, vitae auctor ligula fermentum eu. Cras sit amet urna enim. Sed feugiat ante nec neque ornare suscipit. Nam pretium eget urna nec molestie. In nec pretium enim. Aliquam est dolor, bibendum at tempor ut, volutpat a lectus. Duis ullamcorper diam bibendum lorem maximus, eget lacinia justo feugiat. Aenean sit amet velit et quam consequat interdum euismod ac erat. Ut ut dolor eu orci tincidunt venenatis. Duis turpis felis, venenatis quis efficitur a, tempor ut dolor.\n\nQuisque cursus leo sit amet hendrerit feugiat. Aliquam erat volutpat. Aliquam egestas semper nulla sit amet mollis. Nunc sit amet augue a nisl vestibulum ultrices eu ut turpis. Proin cursus, lorem quis porta scelerisque, ante sem vehicula tellus, in pharetra nulla eros ac dolor. Interdum et malesuada fames ac ante ipsum primis in faucibus. Nullam id convallis diam. Nulla mauris lectus, bibendum in orci quis, blandit tempor erat. Fusce vitae lorem elit.', 
                     'Cancer', 30000)
    db.commit()
finally:
    db.close()

In [15]:
# Create mock sites data
db_meta = get_db(test_db_paths[1])
db_text = get_db(test_db_paths[2])
try:
    insert_mock_journal(db_meta, db_text, '1-1', 1, 
                        "First update",
                        "Short first update.")
    insert_mock_journal(db_meta, db_text, '1-2', 1, 
                        "",
                        "No title second update. Two sentences, but still very, very short.")
    insert_mock_journal(db_meta, db_text, '1-3', 1, 
                        "This title is on the longer side (2020-03-01)",
                        "This post contains HTML line-breaks.<br /><br />Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer consequat placerat ullamcorper. Duis porttitor erat non odio ultricies, quis interdum neque porta. Mauris id enim dictum, egestas dui a, pellentesque arcu. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Maecenas non sollicitudin quam. In imperdiet, magna eu volutpat congue, metus sapien vehicula felis, luctus faucibus mauris urna vitae nulla. Pellentesque ac nisl eget ipsum blandit euismod. Pellentesque ex augue, finibus vel varius sit amet, finibus ac tortor. Curabitur auctor molestie odio sit amet convallis. Morbi vel vestibulum lacus, non pretium est. Suspendisse eget commodo magna. Aenean non varius dui, ut placerat est. Maecenas semper enim eu turpis tristique, in pretium massa bibendum. Fusce in sapien neque. Fusce suscipit feugiat purus, sit amet aliquet quam egestas eget.<br /><br />Mauris scelerisque ante id enim molestie, molestie convallis erat dictum. Etiam eu pellentesque ipsum. Interdum et malesuada fames ac ante ipsum primis in faucibus. Donec a vehicula quam, quis consectetur libero. Aliquam erat volutpat. Integer at porttitor felis, vitae molestie augue. In efficitur orci vitae accumsan sagittis. Pellentesque hendrerit est tellus, aliquam malesuada turpis feugiat ut. Vivamus at lacus eget diam mattis consectetur in eget sapien.<br /><br />Maecenas gravida condimentum semper. Aliquam sed commodo augue. Fusce nec odio mi. Suspendisse aliquet quis arcu et mollis. Mauris id odio finibus, ultrices lorem sit amet, ullamcorper urna. Donec imperdiet sit amet nisl eget vestibulum. Donec ultricies interdum metus, id pretium magna dictum id. Aenean eu turpis ex. Donec in sollicitudin eros.<br /><br />Nulla scelerisque laoreet sem, vel vulputate ex feugiat vel. Vestibulum consequat ante ultrices feugiat varius. Aliquam eleifend metus ac dolor facilisis rhoncus. Sed lacus magna, pulvinar non facilisis vitae, elementum eget purus. Curabitur sollicitudin quam orci. Proin ut luctus sapien, varius volutpat metus. Nulla pretium feugiat lectus vitae maximus. Duis fermentum diam ipsum. Etiam at purus urna. Suspendisse at felis vitae sapien tristique vestibulum a non leo. Sed id eleifend augue. Aenean vitae lorem quam. Phasellus ac justo eu ipsum volutpat rhoncus vitae eu risus.<br /><br />Cras cursus rhoncus elementum. Phasellus ornare venenatis libero, vel malesuada ante tempus id. In hac habitasse platea dictumst. Fusce non leo nisi. Sed malesuada, velit id dapibus euismod, felis nisl sodales tellus, non accumsan libero dui et ligula. Morbi nulla libero, cursus non porta nec, iaculis et nisl. Morbi a condimentum justo. In ac suscipit dolor. Nam magna massa, blandit quis dictum in, suscipit eget erat. Fusce tempus mollis mauris non porttitor.")
    insert_mock_journal(db_meta, db_text, '2-1', 2, 
                        "Long update with title",
                        "This long update contains HTML.<br />Mauris a turpis vel felis elementum fermentum non nec felis. Sed ac porttitor metus. Quisque vel erat vel ligula gravida ultrices ac nec ligula. Pellentesque in placerat mauris. In vitae odio fermentum, malesuada risus id, lacinia nisl. Ut rhoncus augue in risus ornare commodo. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer eu erat scelerisque, tincidunt mauris vitae, imperdiet neque. Nulla id rhoncus urna, vel accumsan arcu.<br />In consequat dolor et dictum pulvinar. Morbi ultricies accumsan dolor, in dictum quam vehicula id. Integer ultricies mattis ligula, eget gravida risus sollicitudin a. Duis tristique velit eu diam pellentesque, tincidunt dapibus ex lobortis. Suspendisse venenatis justo laoreet tincidunt semper. Quisque tempor vulputate congue. Aliquam erat volutpat. Morbi diam augue, luctus in nisl in, efficitur suscipit eros. Nulla facilisi. Nunc quis pellentesque lectus, id gravida augue.<br /><br />Quisque eu risus sit amet nulla lobortis dictum sed vitae tellus. Phasellus sit amet lobortis justo, auctor pulvinar nulla. Etiam bibendum semper lorem in ultricies. Quisque congue metus eu enim pharetra, eu lacinia sem cursus. Maecenas posuere, lectus vitae pharetra ornare, neque metus sodales mi, sit amet suscipit leo mi ac tortor. Aenean sollicitudin accumsan nulla. Proin ut eros nec ligula blandit interdum vel quis turpis.<br />Phasellus nec tellus fringilla, vulputate eros vel, mattis mauris. Donec tincidunt commodo porttitor. Mauris fringilla, augue in semper cursus, turpis mauris eleifend mauris, non maximus elit nulla congue risus. Sed ultricies mauris quis justo porttitor vulputate. Ut lacus justo, commodo ut massa sit amet, luctus dapibus lacus. Maecenas eu suscipit tellus. Aenean ultrices metus dignissim augue tempus, non consequat neque congue. Morbi laoreet eget velit at commodo. Integer at viverra neque.<br /><br />Signature.")
    insert_mock_journal(db_meta, db_text, '2-2', 2, 
                        "",
                        "Very short update, no title.")
    insert_mock_journal(db_meta, db_text, '2-3', 2, 
                        "Long update, with a title that's on the longer side.",
                        "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque commodo, elit id suscipit tristique, orci arcu dictum tortor, suscipit iaculis lorem nisl in lectus. Quisque lobortis vel urna et sollicitudin. Proin nisl nisl, dignissim a varius quis, ornare eget erat. Ut aliquam elit ac tortor rhoncus mollis. Ut sed urna magna. Suspendisse id risus sed nibh porttitor porttitor vestibulum quis ligula. Maecenas sem augue, hendrerit ac erat sodales, varius tempor arcu. Aliquam quis urna fermentum dolor hendrerit varius sed nec est. Donec pharetra ullamcorper mi a convallis.<br /><br />Praesent tempus eros vitae libero bibendum interdum. Praesent ac malesuada arcu. Integer convallis euismod mi. Praesent sagittis mauris quis pretium interdum. Pellentesque feugiat imperdiet dui quis eleifend. Nullam imperdiet tempus rhoncus. Fusce tincidunt elit ligula, eget gravida justo interdum consectetur. Mauris non eros nec lacus rhoncus pharetra. Interdum et malesuada fames ac ante ipsum primis in faucibus. Maecenas semper erat at imperdiet rhoncus. Etiam eu lacus at nunc lobortis pellentesque quis sed enim. Vestibulum sed tincidunt felis.<br /><br />Donec vitae consectetur urna, vel iaculis diam. Aliquam eleifend ex risus, sit amet pulvinar libero hendrerit a. Ut viverra purus nec quam maximus, eget hendrerit erat egestas. Vestibulum condimentum malesuada bibendum. Nullam eleifend tempus tellus a fermentum. Fusce sollicitudin, risus a imperdiet interdum, justo lectus placerat neque, ac aliquam mauris turpis nec magna. Cras iaculis massa ut interdum suscipit. Nunc eleifend odio leo, quis efficitur elit condimentum eu. Donec non tincidunt erat. Curabitur auctor cursus nibh eu sodales. In at nisl nulla.<br />Cras pellentesque gravida turpis, eu aliquet urna vulputate sit amet. Integer molestie nibh a volutpat lobortis. Integer eget ultrices nulla, at viverra mauris. Sed ultricies, ligula quis scelerisque sodales, augue nisl lobortis leo, a mollis nisi sem sit amet turpis. Cras dictum enim nec dui vulputate varius. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Quisque sit amet velit odio. Duis convallis eleifend augue, molestie bibendum eros dignissim ut. Cras quis dui massa. Nullam id congue sem. Nulla volutpat turpis non lobortis tempus.<br />Fusce sollicitudin eleifend lectus sit amet vulputate. Vestibulum et vestibulum leo. Vestibulum euismod lobortis libero quis malesuada. Sed tellus nulla, placerat tempor condimentum vitae, malesuada non nunc. Curabitur lacinia, mi in dictum porta, quam turpis facilisis turpis, id fermentum lacus ex ut sapien. Proin condimentum quam sagittis, consequat purus id, tempus augue. Duis iaculis finibus scelerisque. Cras ut vehicula neque. Nunc ultricies volutpat nunc id rhoncus. Curabitur nisi est, pretium venenatis gravida vitae, tincidunt ac dui. Ut sollicitudin commodo scelerisque. Nam scelerisque varius sollicitudin.")
    # site 3
    insert_mock_journal(db_meta, db_text, f'3-0', 4, 
                    f"Update #0 - guest post",
                    "This is a short initial journal update from a different user than the rest of the site.")
    for i in range(1, 200):
        insert_mock_journal(db_meta, db_text, f'3-{i}', 3, 
                        f"Update #{i}",
                        "Journal update on a site with many updates. Lorem ipsum dolor sit amet, consectetur adipiscing elit.")
    db_meta.commit()
    db_text.commit()
finally:
    db_meta.close()
    db_text.close()



In [16]:
# Create mock comments data
db = get_db(test_db_paths[3])
try:
    insert_mock_comment(db, 1, '1-1', 2, "Test comment from user 2.")
    insert_mock_comment(db, 1, '1-1', 3, "Test comment from user 3.")
    insert_mock_comment(db, 1, '1-2', 3, "Test comment from user 3, a bit longer.  Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis venenatis purus id diam aliquam, et volutpat metus euismod. Cras non neque ut orci vestibulum tristique. Praesent non congue enim. Vivamus et nulla non purus accumsan lobortis finibus in felis. Aliquam molestie tellus quis mauris imperdiet commodo. Vivamus malesuada feugiat dolor, vel ullamcorper erat condimentum vitae. Phasellus laoreet dapibus accumsan. Aliquam vestibulum tempus elit, a suscipit purus. Praesent sodales sapien id metus volutpat, a ornare odio dapibus. Donec iaculis posuere elit, ac vestibulum lectus molestie a. Vivamus vitae augue vitae lorem tempus molestie.")
    insert_mock_comment(db, 1, '1-3', 100, "Test comment from a visitor, very long.  Vivamus dapibus turpis orci, eu imperdiet ante dapibus quis. Vivamus sed ligula nec massa sollicitudin consectetur. Morbi pretium felis odio, cursus interdum dui fermentum eget. Nam a elit et augue pellentesque aliquam. Sed fringilla, magna vel fringilla interdum, ante nibh sagittis sem, in malesuada est enim et sapien. Suspendisse vulputate in orci et mollis. In in mi leo. Duis bibendum lacus ac elit gravida, et sagittis tellus pharetra. Suspendisse hendrerit erat eu tempor sagittis. Donec lacinia in justo nec tempus. Phasellus eget placerat urna, eu bibendum sapien. Quisque pulvinar magna erat. Nunc fringilla id odio ut ornare. Sed convallis eros a metus pretium posuere. Mauris sollicitudin felis turpis, vitae venenatis lorem malesuada ut. Integer ac efficitur felis.\nPellentesque ut nibh laoreet, pretium nunc a, posuere nisl. Nullam lorem dolor, vestibulum nec congue luctus, lobortis a libero. Donec ornare euismod ornare. Cras quis semper ipsum. Integer id velit ut augue rutrum commodo. Nam a quam et ante pulvinar bibendum. Cras et nisi semper leo sagittis molestie et in urna. In hac habitasse platea dictumst. Donec eget rhoncus leo, facilisis ultrices purus. Donec varius, nibh sed viverra tincidunt, justo arcu semper neque, et molestie nibh sem id magna. In id ex nec ipsum pretium viverra at sit amet nisl.\nUt ultricies nec leo et consectetur. Etiam tincidunt risus sed dapibus malesuada. Nunc molestie cursus venenatis. Vivamus libero urna, venenatis pellentesque semper id, blandit non ante. Vivamus ornare lectus mollis purus mollis finibus. Sed viverra egestas metus, a luctus quam feugiat nec. Sed sit amet metus lobortis leo varius sollicitudin a feugiat nibh. Mauris condimentum bibendum metus quis sodales. Donec massa nisi, cursus eget aliquet eget, auctor vel lectus. Nulla nec lectus faucibus, rutrum magna in, posuere nulla. Pellentesque semper tempus scelerisque. Ut non pharetra purus, nec condimentum massa. Donec posuere aliquet fringilla.\nPhasellus vitae est sed odio feugiat finibus. Ut eget ipsum rutrum, facilisis felis id, vehicula ligula. Vivamus malesuada massa dui. Nulla at lacus at augue vestibulum sagittis eget id nisl. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Quisque ullamcorper dolor a nibh porttitor, nec tempor orci tincidunt. Curabitur rhoncus odio sed eros molestie elementum. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Curabitur blandit risus eget massa dictum fermentum. Mauris auctor laoreet consectetur. Fusce blandit scelerisque lacus, ac pulvinar magna ullamcorper a. Aliquam erat volutpat. Etiam pellentesque fermentum lorem, eu facilisis odio pulvinar quis. Vestibulum rutrum nunc augue, in malesuada quam lacinia sed. Duis bibendum viverra ligula, a malesuada diam rhoncus nec. Donec sem leo, consequat et nisi eget, ultrices sodales velit.")
    db.commit()
finally:
    db.close()

In [17]:
# Create mock guestbook data
db = get_db(test_db_paths[4])
try:
    insert_mock_guestbook(db, 1, 2, "Guestbook entry from user 2 on site 1.")
    insert_mock_guestbook(db, 1, 3, "Guestbook entry from user 3 on site 1.")
    insert_mock_guestbook(db, 1, 1, "Guestbook entry from user 1 on site 1.")
    insert_mock_guestbook(db, 2, 1, "Guestbook entry from user 1 on site 2.")
    insert_mock_guestbook(db, 3, 1, "Guestbook entry from user 1 on site 3.")
    insert_mock_guestbook(db, 3, 1, "Second guestbook entry from user 1 on site 3.")
    insert_mock_guestbook(db, 2, 100, 'Guestbook entry from user 100 on site 2.')
    insert_mock_guestbook(db, 3, 101, 'Guestbook entry from user 101 on site 2.')
    db.commit()
finally:
    db.close()

In [18]:
# Create mock amps data
db = get_db(test_db_paths[5])
try:
    insert_mock_amp(db, 1, '1-1', 2)
    insert_mock_amp(db, 1, '1-1', 100)
    insert_mock_amp(db, 1, '1-1', 101)
    insert_mock_amp(db, 1, '1-3', 101)
    db.commit()
finally:
    db.close()

In [19]:
!du -h {test_db_dir}/*.sqlite

64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/amps.sqlite
64K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/guestbook.sqlite
96K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal_metadata.sqlite
160K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/journal.sqlite
32K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/site.sqlite
96K	/panfs/roc/groups/1/lana/levon003/repos/recsys-peer-match/cb_web_app/flask_api/test/data/updated_comments.sqlite
